In [ ]:
from moorcheh_sdk import MoorchehClient, MoorchehError
from app.agents import ScamDetectionAgent
from app.api.schemas import ChatMessage
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import re
import os

In [4]:
load_dotenv('./app/.env')

True

In [5]:
dataset = pd.read_csv('./data/scam-dialogue_all.csv')
dataset = dataset.reset_index()
dataset.head()

,index,dialogue,type,label
0,0,"caller: Hello, is this John Smith? receiver: Y...",ssn,1
1,1,"caller: Hello, my name is Officer Johnson from...",ssn,1
2,2,"caller: Hello, is this Mr. Johnson? receiver: ...",ssn,1
3,3,"caller: Hello, my name is Officer Johnson from...",ssn,1
4,4,"caller: Hello, this is Officer Johnson from th...",ssn,1


In [ ]:
parsed = []
    
for _, rows in dataset.iterrows():
    raw: str = rows['dialogue']
    category = rows['type']
    label = rows['label']
    
    USER = 'receiver:'
    OTHER = 'caller:'
    messages = re.split(f'{USER} | {OTHER}', raw)
    
    cur_sender = OTHER if raw.startswith(OTHER) else USER
    messages_parsed = []
    for message in messages:
        sender_value = 'user' if cur_sender == USER else 'other'
        messages_parsed.append(ChatMessage(content=message, sender=sender_value))
        
        cur_sender = USER if cur_sender == OTHER else OTHER

    parsed.append((messages_parsed, category, label))

parsed_df = pd.DataFrame(parsed, columns=['chat', 'category', 'label'])
        

sender='other' content='caller: Hello, is this John Smith? '
sender='other' content="caller: Hello, my name is Officer Johnson from the Social Security Administration. We've been trying to reach you about a serious issue with your social security number. "
sender='other' content='caller: Hello, is this Mr. Johnson? '
sender='other' content="caller: Hello, my name is Officer Johnson from the Social Security Administration. We've been trying to reach you about a problem with your social security number. "
sender='other' content="caller: Hello, this is Officer Johnson from the Social Security Administration. We've had a report of suspicious activity on your social security number. "
sender='other' content='caller: Hello, is this John Smith? '
sender='other' content='caller: Hello, is this Mr. Johnson?  '
sender='other' content='caller: Hello, is this Mr. Johnson? '
sender='other' content="caller: Hello, this is Officer Johnson from the Social Security Administration. We've been trying to 

In [ ]:
scam_convos = parsed_df[parsed_df['label'] == 1]


os.makedirs('./data/scam_convos', exist_ok=True)
for idx, row in scam_convos.iterrows():
    with open(f'./data/scam_convos/{idx}_scam.txt', mode='w', encoding='utf-8') as f:
        
        messages = '.\n'.join([str(message) for message in row['chat']])
        category = row['category']
        
        val = f"""
This is a document detailing an scam conversation of the following category.
Category: {category}

Conversation:
{messages}

END OF DOC"""
        
        f.write(val)
        


In [27]:
client = MoorchehClient(os.environ.get('MOORCHEH_KEY'))
uploads = []
for file in os.listdir('./data/scam_convos')[1:]:
    
    id = file.removesuffix('.txt')
    content = None
    with open(os.path.join("./data/scam_convos" ,file), mode='r', encoding='utf-8') as f:
        content = f.read(-1)
    uploads.append({'id': id,'text': content, 'type': 'scam_convo'})

res = client.upload_documents('scam_detection', documents=uploads)
print(res)

APIError: API Error (Status: 504): API Error: {"message": "Endpoint request timed out"}

In [30]:
uploads = []
for file in os.listdir('./data/guides')[0:1]:
    
    id = file.removesuffix('.txt')
    content = None
    with open(os.path.join("./data/guides" ,file), mode='r', encoding='utf-8') as f:
        content = f.read(-1)
    uploads.append({'id': id,'text': content, 'type': 'guides'})

res = client.upload_documents('scam_detection', documents=uploads)
print(res)

{'status': 'success', 'message': 'Successfully queued 1 documents for processing.', 'queued_documents': 1, 'document_ids': ['common_scam_types'], 'execution_time': 0.581}
